In [1]:
import os, sys, platform

print(f"python version: {sys.version}")
print(f"os type: {os.name}")
print(f"os: {platform.system()}")
print(f"os release: {platform.release()}")

python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
os type: posix
os: Linux
os release: 5.15.65+


In [2]:
repo_path = "/kaggle/working/hana_mammograph"
if not os.path.isdir(repo_path):
    ! git clone https://github.com/winsoncws/hana_mammograph.git
os.chdir(repo_path)

Cloning into 'hana_mammograph'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 35 (delta 12), reused 22 (delta 5), pack-reused 0
Unpacking objects: 100% (35/35), 10.80 KiB | 1.08 MiB/s, done.


In [3]:
# update custom repo
! git pull

Already up to date.


In [4]:
# List files in input directory
import os
train_count = 0
test_count  = 0
test_patient_count = 0
train_patient_count = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    if dirname.split("/", -1)[-1].isnumeric():
        if 'test_images' in dirname:    
            test_patient_count += 1
        else:
            train_patient_count += 1
    else:
        print(dirname)
    for filename in filenames:
        if "dcm" in filename:
            if 'test_images' in dirname:
                test_count += 1
            else:
                train_count += 1
        else:
            print("    " + filename)

print("total train patients:", train_patient_count)
print("total test patients:", test_patient_count)
print("total train:", train_count)
print("total test :", test_count)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/monai-whl
    monai-1.1.0-202212191849-py3-none-any.whl
/kaggle/input/torchio
    torchio-0.18.86-py2.py3-none-any.whl
    Deprecated-1.2.13-py2.py3-none-any.whl
    shellingham-1.5.0.post1-py2.py3-none-any.whl
/kaggle/input/rsna-breast-cancer-detection
    sample_submission.csv
    train.csv
    test.csv
/kaggle/input/rsna-breast-cancer-detection/train_images
/kaggle/input/rsna-breast-cancer-detection/test_images
total train patients: 11913
total test patients: 1
total train: 54706
total test : 4


In [5]:
import numpy
import matplotlib.pyplot as plt
import pydicom
import cv2
import SimpleITK as sitk
import torch
import torchvision
from torch.utils.data import DataLoader

In [6]:
# pip install packages not in notebook environment
try:
    import torchio as tio
except:
    ! pip install --no-index --find-links=/kaggle/input/torchio torchio
try:
    import torchio
except:
    ! pip install --no-index --find-links=/kaggle/input/monai-whl monai
    
print("all non-standard packages successfully installed and imported.")

Looking in links: /kaggle/input/torchio
Processing /kaggle/input/torchio/torchio-0.18.86-py2.py3-none-any.whl
Processing /kaggle/input/torchio/Deprecated-1.2.13-py2.py3-none-any.whl
Processing /kaggle/input/torchio/shellingham-1.5.0.post1-py2.py3-none-any.whl
all non-standard packages successfully installed and imported.


In [7]:
# import custom module
from isaiah import MammoData

In [8]:
torch.manual_seed(42)
training_data_dir = "/kaggle/input/rsna-breast-cancer-detection/train_images"

In [9]:
ds = MammoData(training_data_dir)
print(len(ds))

54706


In [10]:
train_dataloader = DataLoader(ds, batch_size=1, shuffle=True)